## Overview

#### This note book is a pipline connecting to the git hub repository: https://github.com/CSSEGISandData/COVID-19 and extrating the following data related to the corona virus

1. Country Level Confirmed Cases Per Day
2. Country Level Deaths Per Day

### Terms of use:

This GitHub repo and its contents herein, including all data, mapping, and analysis, copyright 2020 Johns Hopkins University, all rights reserved, is provided to the public strictly for educational and academic research purposes. The Website relies upon publicly available data from multiple sources, that do not always agree. The Johns Hopkins University hereby disclaims any and all representations and warranties with respect to the Website, including accuracy, fitness for use, and merchantability. Reliance on the Website for medical guidance or use of the Website in commerce is strictly prohibited.

#### Import Database Structure
This is the SQL database structure used


In [2]:
from database import model as m

#### Connect to database
RDS SQL Server Data Base

In [3]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import pandas as pd

secret = {
    "username": "admin",
    "password": "",
    "host": "database-1.ccwgqdqrrmvt.eu-west-1.rds.amazonaws.com",
    "port": "1433"
}

engine = create_engine(
    'mssql+pymssql://' +
    secret['username'] + ':' + secret['password'] + '@' + secret['host'] + ':' +
    str(secret['port']) + '/Corona'

)

session = sessionmaker()(bind=engine)

#### Import Data

##### Confirmed_Cases

In [4]:
df_C = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")

##### Deaths

In [ ]:
df_D = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")

#### Clean Data

##### Clean Confirmed Cases

In [4]:
df_C = df_C.melt(id_vars = ["Country/Region","Lat","Long","Province/State"])

df_C_country_level = df_C.groupby(["Country/Region","variable"]).sum()

df_C_country_level = df_C_country_level.reset_index()

df_C_country_level["Date"] = df_C_country_level.variable.apply(lambda x: pd.to_datetime(x))

df_C_country_level = df_C_country_level[df_C_country_level["value"]!=0]

##### Clean Deaths

In [5]:
df_D = df_D.melt(id_vars = ["Country/Region","Lat","Long","Province/State"])

df_D_country_level = df_D.groupby(["Country/Region","variable"]).sum()

df_D_country_level = df_D_country_level.reset_index()

df_D_country_level["Date"] = df_D_country_level.variable.apply(lambda x: pd.to_datetime(x))

df_D_country_level = df_D_country_level[df_D_country_level["value"]!=0]

#### Merge deaths with confirmed cases

In [6]:
df_merge = df_C_country_level.merge(df_D_country_level ,on = ["Country/Region","Date"],how = "left")

#### Cleaning Merged Data

In [7]:
def error(x):
    if pd.isnull(x):
        return 0
    else:
        return x
    
df_merge["deaths"] = df_merge.value_y.apply(lambda x: error(x))

#### Upload data to SQL

In [4]:
for i in range(len(df_merge)):
    
    ## Check Country exisits
    Country = session.query(m.Country).filter(m.Country.country == df_merge["Country/Region"][i]).first()
    if Country is None:
        Country = m.Country(country = df_merge["Country/Region"][i])
        session.add(Country)
        session.commit()
    
    Cases = m.Cases( 
                        date = df_merge["Date"][i],
                        confirmed = int(df_merge["value_x"][i]),
                        deaths = int(df_merge["deaths"][i]),
                        country_id = Country.id
    )
    
    session.add(Cases)
    
session.commit()
session.close()